In [175]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
sb.set()

In [176]:
data = pd.read_csv('categorical.csv')
data = data.drop("Attrition", axis=1)
resp = pd.read_csv("data.csv",usecols = ['Attrition'])


In [177]:
data.head()


BusinessTravel              Department  Education EducationField  \
0      Travel_Rarely                   Sales          2  Life Sciences   
1  Travel_Frequently  Research & Development          1  Life Sciences   
2      Travel_Rarely  Research & Development          2          Other   
3  Travel_Frequently  Research & Development          4  Life Sciences   
4      Travel_Rarely  Research & Development          1        Medical   

   EnvironmentSatisfaction  Gender  JobInvolvement  JobLevel  \
0                        2  Female               3         2   
1                        3    Male               2         2   
2                        4    Male               2         1   
3                        4  Female               3         1   
4                        1    Male               3         1   

                 JobRole  JobSatisfaction MaritalStatus OverTime  \
0        Sales Executive                4        Single      Yes   
1     Research Scientist                2       Married       No   
2  Laboratory Technician                3        Single      Yes   
3     Research Scientist                3       Married      Yes   
4  Laboratory Technician                2       Married       No   

   PerformanceRating  RelationshipSatisfaction  StockOptionLevel  \
0                  3                         1                 0   
1                  4                         4                 1   
2                  3                         2                 0   
3                  3                         3                 0   
4                  3                         4                 1   

   WorkLifeBalance  
0                1  
1                3  
2                3  
3                3  
4                3

In [178]:
resp.head()

Attrition
0       Yes
1        No
2       Yes
3        No
4        No

In [179]:
data = pd.get_dummies(data)

In [180]:
data.head()


Education  EnvironmentSatisfaction  JobInvolvement  JobLevel  \
0          2                        2               3         2   
1          1                        3               2         2   
2          2                        4               2         1   
3          4                        4               3         1   
4          1                        1               3         1   

   JobSatisfaction  PerformanceRating  RelationshipSatisfaction  \
0                4                  3                         1   
1                2                  4                         4   
2                3                  3                         2   
3                3                  3                         3   
4                2                  3                         4   

   StockOptionLevel  WorkLifeBalance  BusinessTravel_Non-Travel  ...  \
0                 0                1                          0  ...   
1                 1                3                          0  ...   
2                 0                3                          0  ...   
3                 0                3                          0  ...   
4                 1                3                          0  ...   

   JobRole_Manufacturing Director  JobRole_Research Director  \
0                               0                          0   
1                               0                          0   
2                               0                          0   
3                               0                          0   
4                               0                          0   

   JobRole_Research Scientist  JobRole_Sales Executive  \
0                           0                        1   
1                           1                        0   
2                           0                        0   
3                           1                        0   
4                           0                        0   

   JobRole_Sales Representative  MaritalStatus_Divorced  \
0                             0                       0   
1                             0                       0   
2                             0                       0   
3                             0                       0   
4                             0                       0   

   MaritalStatus_Married  MaritalStatus_Single  OverTime_No  OverTime_Yes  
0                      0                     1            0             1  
1                      1                     0            1             0  
2                      0                     1            0             1  
3                      1                     0            0             1  
4                      1                     0            1             0  

[5 rows x 37 columns]

In [181]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 37 columns):
 #   Column                             Non-Null Count  Dtype
---  ------                             --------------  -----
 0   Education                          1470 non-null   int64
 1   EnvironmentSatisfaction            1470 non-null   int64
 2   JobInvolvement                     1470 non-null   int64
 3   JobLevel                           1470 non-null   int64
 4   JobSatisfaction                    1470 non-null   int64
 5   PerformanceRating                  1470 non-null   int64
 6   RelationshipSatisfaction           1470 non-null   int64
 7   StockOptionLevel                   1470 non-null   int64
 8   WorkLifeBalance                    1470 non-null   int64
 9   BusinessTravel_Non-Travel          1470 non-null   uint8
 10  BusinessTravel_Travel_Frequently   1470 non-null   uint8
 11  BusinessTravel_Travel_Rarely       1470 non-null   uint8
 12  Department_Human Res

In [194]:
from imblearn.over_sampling import SMOTE 
resp['attrition_binary'] = resp["Attrition"].apply(lambda x: 1 if x == 'Yes' else 0)
LogReg = LogisticRegression(max_iter=1000)
x_train, x_test, y_train, y_test = train_test_split(data, resp['attrition_binary'], test_size=0.25, random_state=42)
oversampler= SMOTE(random_state=0)
x_train, y_train = oversampler.fit_resample(x_train,y_train)
LogReg.fit(x_train, y_train)
train_acc = LogReg.score(x_train, y_train)
y_pred = LogReg.predict(x_test)
test_acc = LogReg.score(x_test, y_test)
print("Accuracy on Train set:", train_acc)
print("Accuracy on Test set:", test_acc)

Accuracy on Train set: 0.9107338444687842
Accuracy on Test set: 0.8614130434782609


In [195]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.94      0.92       320
           1       0.46      0.35      0.40        48

    accuracy                           0.86       368
   macro avg       0.68      0.65      0.66       368
weighted avg       0.85      0.86      0.85       368



In [196]:
# Get the coefficients of the logistic regression model
coefficients = LogReg.coef_[0]

# Get the absolute values of the coefficients
absolute_coefficients = abs(coefficients)

# Rank the variables based on their importance
variable_importance = sorted(zip(absolute_coefficients, data.columns), reverse=True)

# Print the variable importance rankings
print("Variable Importance Rankings:")
for i, (coef, name) in enumerate(variable_importance):
    print("{}. {} ({:.4f})".format(i+1, name, coef))

Variable Importance Rankings:
1. OverTime_No (2.5768)
2. BusinessTravel_Non-Travel (2.4715)
3. EducationField_Other (2.4348)
4. EducationField_Life Sciences (2.4022)
5. EducationField_Medical (2.3427)
6. Gender_Female (2.3327)
7. MaritalStatus_Divorced (2.2731)
8. EducationField_Marketing (2.1255)
9. JobRole_Healthcare Representative (2.0530)
10. Gender_Male (1.9690)
11. JobRole_Research Director (1.8513)
12. MaritalStatus_Married (1.8210)
13. JobRole_Manufacturing Director (1.7477)
14. EducationField_Technical Degree (1.6593)
15. JobRole_Research Scientist (1.6383)
16. BusinessTravel_Travel_Rarely (1.4699)
17. Department_Human Resources (1.4530)
18. JobRole_Manager (1.3133)
19. Department_Research & Development (1.2403)
20. Department_Sales (1.0978)
21. JobRole_Sales Executive (1.0791)
22. BusinessTravel_Travel_Frequently (1.0010)
23. JobRole_Human Resources (0.9878)
24. EducationField_Human Resources (0.9480)
25. OverTime_Yes (0.8661)
26. MaritalStatus_Single (0.8099)
27. JobRole_Lab

From this we can infer that overtime, businessTravel are important factors in determining attrition

In [197]:
resp['attrition_binary'] = resp["Attrition"].apply(lambda x: 1 if x == 'Yes' else 0)
X = data[['OverTime_No','BusinessTravel_Non-Travel']]
y = resp['attrition_binary']
model = LogisticRegression().fit(X, y)

# Create a plot of the logistic regression curve
f, axes = plt.subplots(2, 1, figsize=(24, 12))
sb.regplot(x=X['OverTime_No'], y=y, logistic=True, scatter_kws={'alpha':0.1}, ax=axes[0])
sb.regplot(x=X['BusinessTravel_Non-Travel'], y=y, logistic=True, scatter_kws={'alpha':0.1}, ax=axes[1])


<AxesSubplot:xlabel='BusinessTravel_Non-Travel', ylabel='attrition_binary'>